In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re  
import musicbrainzngs


/var/folders/00/6v4ypdjx7mlb7kcwyj2mg58m0000gn/T/ipykernel_11042/595958206.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [52]:
years = [1960 + i for i in range (0,64)]
old_years = [1945 + i for i in range (0,15)]

In [53]:
link = f"https://en.wikipedia.org/wiki/List_of_Billboard_200_number-one_albums_of_{years[3]}"
response = requests.get(link)
soup = BeautifulSoup(response.text, 'html.parser')


In [54]:
response = requests.get(link)

In [55]:
soup = BeautifulSoup(response.text, 'html.parser')


In [56]:
def get_mono_albums(soup, year, len_column = 3):
    records = []
    table_rows = soup.find_all('tr')[1:]  # Skipping the header row based on your snippet
    for row in table_rows:
        columns = row.find_all('td')
        #mono_album_info = columns.find_all('a')
        if len(columns)> len_column:  
            mono_album_info = columns[0].find('a')
            mono_artist_info = columns[1].text.strip()
            if mono_album_info:
                
                url = "https://en.wikipedia.org" + mono_album_info['href']
                response_image = requests.get(url)
                page_content = response_image.content
                
                # Step 2: Parse the HTML content
                soup_image = BeautifulSoup(page_content, 'html.parser')
                
                # Step 3: Find the infobox within the page
                infobox = soup_image.find('table', class_='infobox')
                full_image_url = None
                genre = None
                
                if infobox:
                    genre_row = infobox.find('th', text=lambda x: x and 'Genre' in x)
                    image_tag = infobox.find('img')
                    if image_tag and 'src' in image_tag.attrs:
                        image_src = image_tag['src']
                        full_image_url = f"https:{image_src}"

                    if genre_row:
                        genre_cell = genre_row.find_next_sibling('td')
                        if genre_cell:
                            
                            genres = genre_cell.find_all('a')
                            genre_names = [ re.sub(r'\[.*?\]', '', genre.text.strip()) for genre in genres if genre.text.strip()]
                            genre_names = [name for name in genre_names if name]  # Filter out empty strings
                            genre = ', '.join(genre_names)

                            genre = re.sub(r'\[.*?\]', '', genre)
                else:
                    full_image_url = None
                    print("Image not found.")
                if year == 1951:
                    genre = "Pop"
    
                record = {
                    'year': year,
                    "rank":None,
                    'artist_name': mono_artist_info,
                    'album_name': mono_album_info.text,
                    "image_url" : full_image_url,
                    'album_link': "https://en.wikipedia.org" + mono_album_info['href'],
                    "genre":genre,
                }
                records.append(record)
    return records

def get_stereo_albums(soup,year):
    records = []
    table_rows = soup.find_all('tr')[1:]  # Skipping the header row based on your snippet
    for row in table_rows:
        columns = row.find_all('td')

        if len(columns)>4:  # To avoid processing header rows
            stereo_album_info = columns[3].find('a')
            stereo_artist_info = columns[4].text.strip()

            if stereo_album_info:
                url = "https://en.wikipedia.org" + stereo_album_info['href']
                response_image = requests.get(url)
                page_content = response_image.content
                
                # Step 2: Parse the HTML content
                soup_image = BeautifulSoup(page_content, 'html.parser')
                
                infobox = soup_image.find('table', class_='infobox')
                
                full_image_url = None
                genre = None
                
                if infobox:
                    genre_row = infobox.find('th', text=lambda x: x and 'Genre' in x)
                    image_tag = infobox.find('img')
                    if image_tag and 'src' in image_tag.attrs:
                        image_src = image_tag['src']
                        # If needed, convert to a full URL
                        full_image_url = f"https:{image_src}"

                    if genre_row:
                        genre_cell = genre_row.find_next_sibling('td')
                        if genre_cell:
                            
                            genres = genre_cell.find_all('a')
                            genre_names = [ re.sub(r'\[.*?\]', '', genre.text.strip()) for genre in genres if genre.text.strip()]
                            genre_names = [name for name in genre_names if name]  # Filter out empty strings
                            genre = ', '.join(genre_names)

                            genre = re.sub(r'\[.*?\]', '', genre)
                else:
                    full_image_url = None
                    print("Image not found.")
        
    
                record = {
                    'year': year,
                    "rank":None,
                    'artist_name': stereo_artist_info,
                    'album_name': stereo_album_info.text,
                    "image_url" : full_image_url,
                    'album_link': "https://en.wikipedia.org" + stereo_album_info['href'],
                    "genre":genre,
                }
                records.append(record)
    return records

In [ ]:
records_end = []  # To store the extracted records

for old_year in old_years:
    if old_year in [1945, 1946, 1947, 1948]:
        link = f"https://en.wikipedia.org/wiki/List_of_Billboard_Best-Selling_Popular_Record_Albums_number_ones_of_{old_year}"

    if old_year in [1949, 1952, 1956, 1957]:
        link= f"https://en.wikipedia.org/wiki/List_of_Billboard_number-one_albums_of_{old_year}"

    if old_year in [1950, 1953, 1954, 1955, 1958,1959]:
        link= f"https://en.wikipedia.org/wiki/List_of_Billboard_200_number-one_albums_of_{old_year}"

    if old_year in [1951]:
        link= f"https://en.wikipedia.org/wiki/List_of_Billboard_Best_Selling_Pop_Albums_number_ones_of_{old_year}"
        
    print("Year: ", link)
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    records_mono = get_mono_albums(soup, old_year, 2)
    records_end.extend(records_mono)
    records_stereo = get_stereo_albums(soup, old_year)

    if records_stereo:
        print(records_stereo)
        records_end.extend(records_stereo)
        

    

In [ ]:
for year in years[:42]:
    link = f"https://en.wikipedia.org/wiki/List_of_Billboard_200_number-one_albums_of_{year}"
    print("Year: ", year)
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    records_mono = get_mono_albums(soup, year)
    records_end.extend(records_mono)
    records_stereo = get_stereo_albums(soup, year)

    if records_stereo:
        print(records_stereo)
        records_end.extend(records_stereo)

In [60]:
df = pd.DataFrame(records_end)
csv_file_path = 'wiki_albums.csv' 
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

In [61]:
billboard_df = pd.read_csv(f"billboard_albums_data.csv",  header=0)
wikipedia_df = pd.read_csv(f"wiki_albums.csv",  header=0)
billboard_df['album_link']= None
billboard_df['genre']= None

In [62]:
df_stacked = pd.concat([wikipedia_df, billboard_df], axis=0)
df_stacked.reset_index(drop=True, inplace=True)


In [63]:
df_sorted = df_stacked.sort_values(by=['album_name', 'artist_name', 'year'], ascending=True)
df_unique = df_sorted.drop_duplicates(subset=['album_name', 'artist_name'], keep='first')


In [64]:
df_stacked = df_unique.sort_values(by=['year','rank'], ascending=True)

In [65]:
csv_file_path = 'year_albums.csv'  # Specify your desired path and file name
df_stacked.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

In [187]:
bill_stacker_df = pd.read_csv(f"year_albums.csv",  header=0)
bill_stacker_df

,year,rank,artist_name,album_name,image_url,album_link,genre
0,1945,NaN,Glenn Miller & His Orchestra,Glenn Miller,https://upload.wikimedia.org/wikipedia/en/thum...,https://en.wikipedia.org/wiki/Glenn_Miller_(19...,"Dance band, Swing"
1,1945,NaN,Bing Crosby,Merry Christmas,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/Merry_Christmas_...,Christmas
2,1945,NaN,Bing Crosby,Selections from Going My Way,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/Selections_from_...,Popular
3,1945,NaN,Original cast,Song of Norway,NaN,https://en.wikipedia.org/wiki/Song_of_Norway,NaN
4,1945,NaN,The King Cole Trio,The King Cole Trio,https://upload.wikimedia.org/wikipedia/en/thum...,https://en.wikipedia.org/wiki/The_King_Cole_Tr...,Jazz
...,...,...,...,...,...,...,...
3317,2023,188.0,Quavo & Takeoff,Only Built For Infinity Links,https://charts-static.billboard.com/img/2022/1...,NaN,NaN
3318,2023,192.0,Lil Wayne,I Am Music,https://charts-static.billboard.com/img/2023/0...,NaN,NaN
3319,2023,193.0,Jimin,FACE (EP),https://charts-static.billboard.com/img/2023/0...,NaN,NaN
3320,2023,195.0,SEVENTEEN,SEVENTEEN 10th Mini Album: FML (EP),https://charts-static.billboard.com/img/2023/0...,NaN,NaN


In [123]:
musicbrainzngs.set_useragent("YourAppName", "0.1", "YourContactInfo")

def get_metadata_from_album(row, out ):
    
    album_name = row['album_name']
    artist_name = row['artist_name']

    if pd.isna(row['genre']):
        album_genre = None
    else:
        album_genre = row['genre'] 
        
    artist_list = musicbrainzngs.search_artists(query=artist_name)
    artist_info = artist_list['artist-list'] [0] 
    tags_artist = artist_info.get('tag-list', [])
    highest_count_tag_artist = None
    
    if tags_artist:
        
        highest_count_tag_artist = max(tags_artist, key=lambda tag: int(tag['count']))['name']
        
    artist_country = artist_info.get('country', 'N/A')
    artist_gender = artist_info.get('gender', 'N/A')

    result = musicbrainzngs.search_releases(artist=artist_name, release=album_name)
    
    release = result['release-list'][0]
    
    genres = release.get('tag-list', [])
    
    if album_genre:
        #print(' album_genre')
        first_three_genres = album_genre
        
    else:
        #print('not else')
        if genres:
            first_three_genres = [genre['name'] for genre in genres[:3]]
            first_three_genres = ', '.join(first_three_genres)
        else:
            release_second = result['release-list'][1]
            
            genres_second = release_second.get('tag-list', [])
            if genres_second:
                first_three_genres = [genre['name'] for genre in genres_second[:3]]
                first_three_genres = ', '.join(first_three_genres)
            else:
                #print("no luck")
                first_three_genres = None

    if out ==1:
        return artist_country
    elif out == 2:
        return artist_gender
    elif out == 3:
        return first_three_genres
    else:
        return highest_count_tag_artist


In [ ]:
bill_stacker_df['artist_country'] = bill_stacker_df.apply(lambda row: get_metadata_from_album(row, 1), axis=1)


In [ ]:
bill_stacker_df['artist_gender'] = bill_stacker_df.apply(lambda row: get_metadata_from_album(row, 2), axis=1)
bill_stacker_df['first_three_genres'] = bill_stacker_df.apply(lambda row: get_metadata_from_album(row, 3), axis=1)
bill_stacker_df['highest_count_tag_artist'] = bill_stacker_df.apply(lambda row: get_metadata_from_album(row, 4), axis=1)


In [492]:
def get_gender_from_album(row):
    
    not_genres = {
            "artist": None,
            "video": None,
            "british":None,
            "self": None,
            "missing":None,
            "french": None,
            "group": None,
            "film":"Soundtrack",
            "death by":None,
            "2008 universal":None,
            "asia version":None,
            "isrc":None,
            "2007/01":None,
            "http":None,
            "audio bluray":None,
            "american":None,
            "discogs/>$100":None,
            "russian": None,
            "bonus track":None,
            "_wrong":None,
            "officialjeffthe_kid":None,
            "cccd": None,
            "private/":None,
            "todo fix recordings":None,
            "mbs-8393": None,
            "ccm":None,
            "good":None,
            "usa":None,
            "cover art error":None,
            "decca records": None,
            "canadian":None,
            "reissue":None,
            "audiobook":None,
            "playlist":None,
            "cd extra":None,
            "queer":None,
            "songwriter":None,
            "cover mount":None,
            "a cappella":None,
            "liste de courses jpn":None,
            "multiple release dates":None,
            "checkme":None,
            "linkin park":None,
            "download code":None,
            "rihanna":None,
            "animated cover art":None,
            "c 7 da m":None,
            "tim mcgraw":None,
            "streaming":None,
            "nickelback":None,
            "mold of tree":None,
            "fan of a fan the album":None,
            "cd-text":None,
            "multiple packaging":None,
            "misunderstood":None,
            "gaga":None,
            "minneapolis":None,
            "cover art":None,
            "grammy":None,
            "toronto":None,
            "Djcjnr #00000#":None,
            "Hidden Track":None,
            "Hdtracks":None,
            "Madrid":None,
            "Scuzk6Obi8Nkj3Dhyqzkrzqwkva":None,
            "Conductor":None,
            "Film Score":None,
        "Ccm":None,
                }

    album_name_hints = {
                "Soundtrack": "Soundtrack",
                    }
    if pd.isna(row['genre']):
        genre = None
    else:
        genre = row['genre'] 

    
    if pd.isna(row['first_three_genres']):
        first_three_genres = None
    else:
        first_three_genres = row['first_three_genres'] 

    if pd.isna(row['highest_count_tag_artist']):
        highest_count_tag_artist = None
    else:
        highest_count_tag_artist = row['highest_count_tag_artist'] 

    #if not genre:
        #print("genre ", genre)
        #print("first_three_genres ", first_three_genres)
        #print("highest_count_tag_artist ", highest_count_tag_artist)
        #print(row['album_name'], row['artist_name'])
        
    if not genre:
        if not first_three_genres:
            
            if highest_count_tag_artist:
                
                output_genre = highest_count_tag_artist
                
                for n_genres in not_genres:
                    if n_genres.lower() in highest_count_tag_artist.lower():
                        #print(not_genres[n_genres])
                        output_genre = not_genres[n_genres]
            else:
                output_genre = None
                
                    
        else:
            output_genre = first_three_genres
            
            for n_genres in not_genres:
                # Check if the first_three_genres is not actually a gender
                if n_genres.lower() in first_three_genres.lower():
                    #if its not a gender then the artist tag is taken into account
                    if highest_count_tag_artist:
                        output_genre = highest_count_tag_artist
                        for n_genres in not_genres:
                            if n_genres.lower() in highest_count_tag_artist.lower():
                                output_genre = not_genres[n_genres]
                    else:
                        output_genre = None

    else:
        output_genre = genre

    alb_name = row['album_name']
    art_name = row['artist_name']
    if not output_genre:
        
        
        for album_name_hint in album_name_hints:
            #print("LOOKING AT ALBUM NAME",  alb_name.lower())
            if album_name_hint.lower() in alb_name.lower():
                output_genre = album_name_hints[album_name_hint]

            #print("LOOKING AT ARTIST NAME", art_name.lower())
            if album_name_hint.lower() in art_name.lower():
                output_genre = album_name_hints[album_name_hint]
        
    if art_name.lower() == 'one direction':
        output_genre = 'pop'
                    
    if output_genre:
        output_genre = output_genre.title()
    #print("output_genre ", output_genre)
    return output_genre

   

    
        
    

In [ ]:
df = bill_stacker_df

In [ ]:
df.loc[9, 'image_url'] = "https://i.ebayimg.com/images/g/yAIAAOSwUsVgvXlH/s-l1600.jpg"
df.loc[11, 'image_url'] = "https://i.discogs.com/lbLeF_zyrQp5kGPtCwwu8CJO58NrXnxcmNPLOhhpN9I/rs:fit/g:sm/q:90/h:600/w:593/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTEwMTEz/NzczLTE0OTE4NDg2/OTItMzAzMy5qcGVn.jpeg"
df.loc[62, 'image_url'] = "https://en.wikipedia.org/wiki/Sing_Along_with_Mitch_(album)#/media/File:Sing_Along_with_Mitch_(album).png"
df.loc[64, 'image_url'] = "https://en.wikipedia.org/wiki/Tchaikovsky_Piano_Concerto_No._1_%28Van_Cliburn_1958_recording%29#/media/File:Van_Cliburn_Tchaikovsky_Piano_Concerto_No.1.jpg"
df.loc[80, 'image_url'] =  "https://i.discogs.com/96Jj55TER_rluxO3pMOKsV7cDLiN60gyThJ9Z1bHMm0/rs:fit/g:sm/q:90/h:600/w:597/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTI3NjU3/MjUtMTU0ODgwNDk3/MC0yMTE4LmpwZWc.jpeg"
df.loc[84, 'image_url'] = "https://i.discogs.com/-51Qu30o6l-KMp8rdA79iBWxFidTvHjLUOxzrxwCVvs/rs:fit/g:sm/q:90/h:600/w:559/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTIyMTEx/OTAtMTI5MzUwOTE4/My5qcGVn.jpeg"
df.loc[328, 'image_url'] = "https://i.discogs.com/wCxgxaDa1IPrphOtQDkg21chN-WCJmhWZ9kDqMTirC8/rs:fit/g:sm/q:90/h:300/w:300/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTQ4Mzc5/MTAtMTM4MTM0NTEw/NC05NjM2LmpwZWc.jpeg"
df.loc[487, 'image_url'] = "https://www.filmico.tv/wp-content/uploads/2023/01/R-6187191-1475781532-2644.jpg"
df.loc[755, 'image_url'] = "https://en.wikipedia.org/wiki/Ryde_or_Die_Vol._3:_In_the_%22R%22_We_Trust#/media/File:RuffRyders-Rydeordievol3.jpg"
df.loc[1415, 'image_url'] = "https://en.wikipedia.org/wiki/Eminem_Presents:_The_Re-Up#/media/File:Eminem_Presents_the_Re-Up.jpg"
df.loc[1626, 'image_url'] = "https://en.wikipedia.org/wiki/Para_Siempre#/media/File:Para_Siempre_-_Vicente_Fern%C3%A1ndez.jpg"
df.loc[1799, 'image_url'] = "https://i.discogs.com/Mq9NmsMA4GGlkL27H5B3q0RudR9bMwmXFt8ym64jhRk/rs:fit/g:sm/q:90/h:600/w:600/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTUwMjU0/NDQtMTQxNDcxMjU5/OS0yMDg3LmpwZWc.jpeg"
df.loc[2104, 'image_url'] = "https://en.wikipedia.org/wiki/Self_Made_Vol._1#/media/File:Self_Made_Vol_1.jpg"
df.loc[2159, 'image_url'] = "https://en.wikipedia.org/wiki/Cruel_Summer_(GOOD_Music_album)#/media/File:CruelSummercover.jpg"
df.loc[2181, 'image_url'] = "https://m.media-amazon.com/images/I/71vA+qmdm9L._SX466_.jpg"
df.loc[2470, 'image_url'] = "https://en.wikipedia.org/wiki/Storyline_%28Hunter_Hayes_album%29"
df.loc[2798, 'image_url'] = "https://en.wikipedia.org/wiki/Sing:_Original_Motion_Picture_Soundtrack#/media/File:Sing_Original_Motion_Picture_Soundtrack.jpeg"
df.loc[2970, 'image_url'] ="https://en.wikipedia.org/wiki/Simi_(mixtape)#/media/File:Simi_Cover.jpeg"
df.loc[3037, 'image_url']="https://m.media-amazon.com/images/I/514gKvJmoUL._UX716_FMwebp_QL85_.jpg"

#df = df.drop([796,940,2861, 2841,2859,1097, 1444, 1877, 1987, 1988, 2009, 2120, 2128, 2135, 2140, 2725, 2845, 2858, 3,2838, 905, 2081, 2374, 2720])

df = df.reset_index().rename(columns={'index': 'index_column'})

In [ ]:
def download_image(image_url, local_file_name):
    headers = {'User-Agent': 'Sele/0.1 (selene.andradelopez@gmail.com)'}
    response = requests.get(image_url, headers=headers)
    #response = requests.get(image_url)
    if response.status_code == 200:
        with open(local_file_name, 'wb') as file:
            file.write(response.content)
        #print(f"Image downloaded successfully: {local_file_name}")
    else:
        print(f"Error: Failed to download image. Status code: {response.status_code}")
        
name = []
for index, row in df.iterrows():
    
    image_name = row['image_name']
    if pd.isna(image_name):
        image_name = None
        
    year = row['year']
    file_name = f"{year}_{index}.png"
    image_url = row['image_url']
    if pd.isna(image_url):
        image_url = None

    if pd.isna(image_url):
        image_url = None
    if image_url:
        
        local_file_name = f"images/{file_name}"
        
        if "https://www.billboard.com" in image_url:
            print(row['album_name'], ' year ', year, image_url)
            file_name = None
        else:
            
            if not image_name:
                print(row['album_name'], ' year ', year, image_url)
                out = download_image(image_url, local_file_name)
        if out == False:
            file_name = None
    else:
        file_name = None
    name.append(file_name)

df['image_name'] = name



In [558]:
df.to_csv("meta_data_billboard.csv", index=False, encoding='utf-8-sig')

## Stats on the data

#https://legismusic.com/es/generos-musicales/
https://www.discogs.com/developers

### Utils

In [559]:

def get_unified_genre(row):
    genres_mappings = {
        # Pop genre mappings
        "Pop Rock": "Pop",
        "Pop,": "Pop",
        ", Pop": "Pop",
        "Pop Rap": "Pop",
        "Pop Punk": "Pop",
        "Indie Pop": "Pop",
        "Dance-Pop": "Pop",
        "Alternative Pop":"Pop",
        "Electropop":"Pop",
        "Synthpop":"Pop",
        "Folk Pop":"Pop",
        "Art Pop":"Pop",
        "Teen Pop":"Pop",
        "Traditional Pop":"Pop",
        "Vocal Pop":"Pop",
        "Glitch Pop":"Pop",
        "Pop And Chart":"Pop",
        "Classic Pop":"Pop",
        "Contemporary Pop":"Pop",
        "Exotica":"Pop",
        "Popular":"Pop",
        
        # Rock genre mappings
        "Alternative Rock": "Rock",
        "Hard Rock": "Rock",
        "Indie Rock": "Rock",
        "Blues Rock": "Rock",
        "Soft Rock": "Rock",
        "Folk Rock": "Rock",
        "Jazz Rock": "Rock",
        "Christian Rock": "Rock",
        "Punk Rock": "Rock",
        "Progressive Rock": "Rock",
        "Rock,": "Rock",
        ", Rock": "Rock",
        "Acoustic Rock": "Rock",
        "Grunge": "Rock",
        "Post-Grunge": "Rock",
        "Emo":"Rock",
        "Punk":"Rock",
        "Southern Rock":"Rock",
        "Indie":"Rock",
        "Surf Rock":"Rock",
        "Roots Rock":"Rock",
        "Art Rock":"Rock",
        "Industrial Rock":"Rock",
        "Melodic Hardcore":"Rock",
        "Anti-Folk":"Rock",
        "Rockabilly":"Rock",
        "Swamp Rock":"Rock",
        "Stoner Rock":"Rock",
        "Psychedelic Rock":"Rock",
        "Heartland":"Rock",
        "Psychedelia":"Rock",
        "Lo-Fi":"Rock",
        "Merseybeat":"Rock",
        "Rock/Us":"Rock",
        
        # Rap genre mappings
        "Gangsta Rap": "Rap",
        "Rap,": "Rap",
        ", Rap": "Rap",
        "Drill": "Rap",
        
        # Hip Hop genre mappings
        "trap": "Hip Hop",
        "Hip Hop,": "Hip Hop",
        " Hip Hop ": "Hip Hop",
        ", Hip Hop": "Hip Hop",
        "Hip-Hop": "Hip Hop",
        "Alternative Hip Hop": "Hip Hop",
        "East Coast Hip Hop": "Hip Hop",
        "Teddys Jams": "Hip Hop",
        "Hardcore": "Hip Hop",
        "Southern Hip Hop": "Hip Hop",
        "Conscious Hip Hop": "Hip Hop",
        "Golden Age Hip Hop": "Hip Hop",
        
        # Country genre mappings
        "Country Pop": "Country",
        "Country,": "Country",
        ", Country": "Country",
        "Modern Country": "Country",
        "Contemporary Country": "Country",
        "Country Rock": "Country",
        "Bluegrass": "Country",
        "Alternative Country": "Country",
        "Neotraditional Country": "Country",
        
        # Metal genre mappings
        ", Metal": "Metal",
        "Metal,": "Metal",
        "Heavy Metal": "Metal",
        "Nu Metal": "Metal",
        "Alternative Metal": "Metal",
        "Symphonic Metal": "Metal",
        "Metalcore":"Metal",
        "Industrial Metal":"Metal",
        "Glam Metal":"Metal",
        "Thrash Metal":"Metal", 
        "Doom Metal":"Metal", 
        "Progressive Metal":"Metal", 
        "Groove Metal":"Metal", 
        
        # RnB genre mappings
        "Contemporary R&B": "RnB",
        ", R&B": "RnB",
        "R&B,": "RnB",
        "RnB,": "RnB",
        ", RnB": "RnB",
        "Alternative R&B": "RnB",
        
        # Electronic genre mappings
        "Electronic,": "Electronic",
        ", Electronic": "Electronic",
        "Electro House": "Electronic",
        "Dubstep":"Electronic",
        "Garage":"Electronic",
        "House":"Electronic",
        "Brostep electronic":"Electronic",
        "Electronica":"Electronic",
        "Dance":"Electronic",
        "Electro":"Electronic",
        "Brostep":"Electronic",
        "Italo-Disco":"Electronic",
        "Techno":"Electronic",
    
        # Soul genre mappings
        "Neo Soul":"Soul",
        "Neo-Soul":"Soul",
        ", Soul":"Soul",
        "Soul,":"Soul",
        "Progressive Soul":"Soul",
    
        # Christmas Music
        "Christmas Music":"Christmas",
    
        # Childrens Music
        "Childrens Music":"Childrens",
        "Children":"Childrens",
    
        # World music
        "New Age":"World Music",
        "Celtic":"World Music",
        "New Wave":"World Music",
    
        # Folk
        "Folk,":"Folk",
        ", Folk":"Folk",
        "Progressive Folk":"Folk",
        "Country Folk":"Folk",
    
        # Classical
        "Classical Crossover":"Classical",
        "Classical,":"Classical",
        ", Classical":"Classical",
        "Opera":"Classical",
        "Classical Voice":"Classical",
        "Choir":"Classical",
    
        # Jazz
        "Vocal Jazz":"Jazz",
        "Jazz,":"Jazz",
        ", Jazz":"Jazz",
        "Smooth Jazz":"Jazz",
        "Bossa Nova":"Jazz",
    
        # Blues
        "Gospel":"Blues",
    
        # Comedy
        "Comedy Music":"Comedy",
        "Stand-Up Comedy":"Comedy",
        "Comedy Rap":"Comedy",
    
        # Christian
        "Praise & Worship":"Christian",
        "Contemporary Christian":"Christian",
        "Christian Metal":"Christian",
        "ccm":"Christian",
    
        # Latin
        "Latin Rock":"Latin",
        "Latin Pop":"Latin",
        "Bachata":"Latin",
        "Latin Trap":"Latin",
        "Salsa":"Latin",
        "Latin":"Latin",
        ", Reggaeton":"Latin",
        "Regional Mexicano":"Latin",
        
        # Rock And Roll 
        ", Rock And Roll":"Rock And Roll",
        "Rock And Roll,":"Rock And Roll",
        "Texican Rock And Roll":"Rock And Roll",
        "Rock & Roll":"Rock And Roll",
        "Rock And Roll":"Rock And Roll",
        
    
        # Funk
        "Funk,":"Funk",
        ", Funk":"Funk",
    
        #Reggae
        "Dancehall":"Reggae",
        "War":"Reggae",
    
        # Broadway
        ", Broadway":"Show Tunes",
        "Musical":"Show Tunes",
    
        # Soundtrack
        ", Soundtrack":"Show Tunes",
        "Film Soundtrack":"Show Tunes",
        "Film Score" : "Show Tunes",
    
        # Easy Listening,
        "Easy Listening,":"Easy Listening",
        "Easy-Listening":"Easy Listening",
    
        # African
        "Calypso":"African",

        
    }
    
    
    genres = {
        "Pop":"Pop",
        "Rock":"Rock",
        "Hip Hop":"Hip Hop",
        "Country":"Country",
        "Soundtrack":"Show Tunes",
        "R&B":"RnB",
        "rnb":"RnB",
        "Jazz":"Jazz",
        "Electronic": "Electronic",
        "Rap": "Rap",
        "Reggaeton":"Latin",
        "Classical":"Classical",
        "Metal":"Metal",
        "Soul":"Soul",
        "Folk":"Folk",
        "K-Pop":"K-Pop",
        "Comedy":"Comedy",
        "Latin":"Latin",
        "Disco":"Disco",
        "Funk":"Funk",
        "Easy Listening":"Easy Listening",
        "Christmas":"Christmas",
        "Reggae":"Reggae",
        "Ballad":"Ballad",
        "Rock And Roll":"Rock And Roll",
        "Vocal" : "Vocal",
        "Mood Music" : "Mood Music",
        "Show Tunes":"Show Tunes",

        # Extra mappings
        "Pop/Rock" : "Pop, Rock",
        "Soul And Reggae" : "Reggae, Soul",
        "Crunk":"Hip Hop",
        "Beat Tapes, New Jersey, Beats":"Pop, Show Tunes"
    }

    sub_genre =  row['sub_genres']
    #print("--------- ----------- ----")
    #print('sub_genres ',sub_genre )

    if pd.isna(sub_genre):
        sub_genre = None
    
    flags = [] 
    genre = ''
    if sub_genre:
        for genre in genres:
            if genre.lower() == sub_genre.lower():
                #print('Unique genre ',sub_genre)
                flags.append(genre)
                

        for g in genres_mappings:
            if g.lower() in sub_genre.lower():
                flags.append(genres_mappings[g])
        
        result_string = ", ".join(sorted(set(flags)))

        
        if len(flags) == 0:
            result_string = '-CHECK-'
            
        #print('result_string ',result_string)
    else:
        result_string = None
    return result_string

In [560]:
def post_processing_genre(row):
    
    extra_genres = {
        "Original cast" : "Show Tunes",
        "I Am Me":"Pop",
        "Autobiography":"Pop",
        "Private Room":"Hip Hop, Rap",
        "The Battle For Everything":"Pop",
        "Bad Boy's 10th Anniversary... The Hits":"Various Artists",
        "WOW Hits 2004":"Various Artists",
        "MONKEY BUSINESS":"Pop",
        "Let's Get It: Thug Motivation 101":"Hip Hop",
        "Totally Country Vol. 4":"Various Artists",
        "WOW Hits 2005":"Various Artists",
        "The Right To Bare Arms":"Comedy",
        "Various Artists":"Various Artists",
        "State Fair" : "Show Tunes",
        "Dorothy Shay Sings":"Show Tunes",
        "South Pacific":"Show Tunes",
        "Cinderella":"Show Tunes",
        "I'm in the Mood for Love" :"Show Tunes",
        "With a Song in My Heart":"Show Tunes",
        "The Student Prince":"Classical",
        "Sing Along with Mitch":"Jazz, Pop",
        "Persuasive Percussion":"Jazz, Pop",
        "Stereo 35/MM":"Jazz, Pop",
        "Now 4" : "Electronic, Pop, Rock",
        "God Bless America: For the Benefit of the Twin Towers Fund" : "Various Artists",
        "Now That's What I Call Music! 6":"Electronic, Pop, Rock",
        "Now That's What I Call Music! 7":"Hip Hop, Pop, Rock",
        "Now 8":"Electronic, Pop, Rock",
        "Now 9" :"Pop",
        "Now 10":"Electronic, Pop, Rock",
        "*NSYNC":"Pop",
        "Totally Hits 2001":"Hip Hop, Pop, Rock",
        "b2k":"RnB",
        "Irv Gotti Presents The Inc":"Hip Hop",
        "Now 7" : "Electronic, Pop, Rock",
        "Ecstasy" : "Hip Hop",
        "America: A Tribute To Heroes":"Various Artists",
        "Oh Aaron":"Pop",
        "A Gangster And A Gentleman":"Hip Hop",
        "God Bless America" :"Various Artists",
        "Dirty Vegas":"Electronic",
        "WOW Hits 2002":"Pop, Rock",
        "The Concert For New York City":"Pop, Rock",
        "WWF: Forceable Entry":"Hip Hop, Rock",
        "Now 11" : "Electronic, Pop, Rock",
        "Now 12" : "Electronic, Pop, Rock",
        "Measure Of A Man":"Pop",
        "Now 13" : "Electronic, Pop, Rock",
        "The Neptunes Present... Clones" : "Hip Hop",
        "Worship Together: I Could Sing Of Your Love Forever":"Various Artists",
        "200 KM/H In The Wrong Lane":"Pop",
        "To Whom It May Concern":"Pop, Rock",
        "Dru World Order" :"RnB",
        "Birdman":"Hip Hop, RnB",
        "Now 14":"Electronic, Pop, Rock",
        "Disneymania: Superstar Artists Sing Disney ...Their Way!":"Show Tunes",
        "WOW Hits 2003":"Pop, Rock",
        "Jagged Edge" : "RnB",
        "Now 15" : "Electronic, Pop, Rock",
        "Hurt No More":"Hip Hop",
        "Now 16" : "Electronic, Pop, Rock",
        "Now 17" : "Electronic, Pop, Rock",
        "Now 18" : "Electronic, Pop, Rock",
        "Now 19" : "Electronic, Pop, Rock",
        "Now 20" : "Electronic, Pop, Rock",
        "Now 21" : "Electronic, Pop, Rock",
        "Now 22" : "Electronic, Pop, Rock",
        "Now 23" : "Electronic, Pop, Rock",
        "Now 24" : "Electronic, Pop, Rock",
        "Now 25" : "Electronic, Pop, Rock",
        "Now 26" : "Electronic, Pop, Rock",
        "Now 27" : "Electronic, Pop, Rock",
        "Now 28" : "Electronic, Pop, Rock",
        "Now 29" : "Electronic, Pop, Rock",
        "Now 30" : "Electronic, Pop, Rock",
        "Now 31" : "Electronic, Pop, Rock",
        "Now 32" : "Electronic, Pop, Rock",
        "Now 33" : "Electronic, Pop, Rock",
        "Now 34" : "Electronic, Pop, Rock",
        "Now 35" : "Electronic, Pop, Rock",
        "Now 36" : "Electronic, Pop, Rock",
        "Now 37" : "Electronic, Pop, Rock",
        "Now 38" : "Electronic, Pop, Rock",
        "Now 39" : "Electronic, Pop, Rock",
        "Take It All Away" : "Pop, Rock",
        "I Need An Angel":"Soul",
        "Retaliation":"Comedy",
        "DANITY KANE":"Hip Hop, Pop",
        "A Thousand Different Ways":"Pop",
        "Wherever You Are":"Rock",
        "Director":"RnB",
        "Taylor Hicks":"Pop",
        "Small Town Girl":"Country",
        "Every Second Counts":"Pop, Rock",
        "Elliott Yamin":"Pop, RnB",
        "Bucky Covington":"Country",
        "The Recession":"Hip Hop, Rap",
        "Rough Around The Edges: Live From Madison Square Garden":"Comedy",
        "DAY26" : "RnB",
        "Scars & Souvenirs":"Rock",
        "Julianne Hough":"Country",
        "Revelation":"Christian, Rock",
        "David Archuleta":"Pop",
        "Kellie Pickler" : "Country",
        "ISolated INcident" : "Comedy",
        "Hit Man: David Foster & Friends" : "Country, Pop",
        "God Willin' & The Creek Don't Rise" : "Rock",
        "Christmas Cheers":"Christmas",
        "The Beginning":"Pop",
        "Barton Hollow":"Folk, Country",
        "TM:103: Hustlerz Ambition" : "Hip Hop",
        "The Lumineers":"Rock",
        "The Civil Wars":"Folk",
        "Best Days":"Blues",
        "Duck The Halls: A Robertson Family Christmas":"Blues, Folk",
        "Three Kings":"Hip Hop",
        "Miracle":"Rock",
        "Don't Kill The Magic":"Pop, Rock",
        "Black Messiah":"Blues, Soul",
        "Pentatonix":"Pop, Rock",
        "Cleopatra":"Folk",
        "Swaay (EP)":"Pop",
        "Nathaniel Rateliff & The Night Sweats" : "Rock, Soul",
        "us (EP)":"Pop",
        "The Intro (EP)":"Pop",
        "Untamed":"Country",
        "PARTYNEXTDOOR 3 (P3)":"Hip Hop",
        "Long Live Nut":"Hip Hop",
        "EVERYTHING IS LOVE":"Hip Hop",
        "Freudian":"Blues",
        "I Am You":"Hip Hop",
        "We All Shine":"Hip Hop",
        "Melly vs. Melvin":"Hip Hop",
        "Homesick (EP)":"Pop",
        "Young Stoner Life: Slime Language 2":"Hip Hop",
        "Purgatory":"Country",
        "The Mockingbird & The Crow":"Country, Rock",
        "5-Star":"K-Pop",
        "Desvelado":"Latin",
        "Danado":"Latin",
        "Rave & Roses":"African",
        "Been One":"Hip Hop",
        "Only Built For Infinity Links":"Hip Hop",
        
    }

    genre_mapping = {
        "R&B":"RnB",
        "rnb":"RnB",
        "Reggaeton":"Latin",
        "Crunk":"Hip Hop",
        "Pop/Rock":"Pop, Rock",
        "Soul And Reggae":"Reggae, Soul",
        "Soundtrack":"Show Tunes",
        "Beat Tapes, New Jersey, Beats":"Pop, Show Tunes",
    }
    if row['big_genres'] == None:
        output_genre = None
        for album_name_hint in extra_genres:
            
            if album_name_hint.lower() == row['album_name'].lower():
                #print(album_name_hint, ' ', row['album_name'])
                output_genre = extra_genres[album_name_hint]

            if album_name_hint.lower() == row['artist_name'].lower():
                #print(album_name_hint, ' ', row['artist_name'])
                output_genre = extra_genres[album_name_hint]

    else:
        output_genre = row['big_genres']
        
        for g_map in genre_mapping:
            if g_map.lower() == output_genre.lower():
                output_genre = genre_mapping[g_map]
        
    
    return output_genre

### Transforming data

In [561]:
df = pd.read_csv(f"meta_data_billboard.csv",  header=0)
df['sub_genres'] = df.apply(lambda row: get_gender_from_album(row), axis=1)

df['big_genres'] = df.apply(lambda row: get_unified_genre(row), axis=1)
df['big_genres'] = df.apply(lambda row: post_processing_genre(row), axis=1)

df.to_csv("meta_data_billboard.csv", index=False, encoding='utf-8-sig')

In [568]:
genres_series = df['big_genres'].str.split(', ').explode()

unique_genres = genres_series.unique()
unique_genres_df = pd.DataFrame(unique_genres, columns=['Unique Genres'])
unique_genres_df.sort_values(by='Unique Genres', inplace=True)
unique_genres_df.reset_index(drop=True, inplace=True)

genres_list = unique_genres_df['Unique Genres'].tolist()

unique_genres_df


,Unique Genres
0,African
1,Ballad
2,Blues
3,Childrens
4,Christian
5,Christmas
6,Classical
7,Comedy
8,Country
9,Disco


In [563]:
value_counts = df['sub_genres'].value_counts(dropna=False)
value_counts_df = value_counts.reset_index()
value_counts_df['Percentage'] = round((value_counts_df['count'] * 100) / 3322, 3)
value_counts_df


,sub_genres,count,Percentage
0,Hip Hop,481,14.479
1,Pop,370,11.138
2,Country,313,9.422
3,None,182,5.479
4,Rock,121,3.642
...,...,...,...
577,"Rock, Pop, Funk, Soul, Synth-Pop, Dance",1,0.030
578,"Contemporary Folk, Folk Rock, Roots Rock",1,0.030
579,"Rock, Blues, Pop, Blue-Eyed Soul, Adult Contem...",1,0.030
580,"Glam Metal, Hard Rock",1,0.030


In [564]:
genre_counts = {genre: 0 for genre in genres_list}

# Iterate through each row of the dataframe
for index, row in df.iterrows():
    # This assumes that each cell in 'big_genres' is a string of genres separated by commas
    genres = row['big_genres'].split(',')
    # Count each genre
    for genre in genres:
        genre = genre.strip()  # Remove any leading/trailing whitespace
        if genre in genre_counts:
            genre_counts[genre] += 1

genre_counts_df = pd.DataFrame(list(genre_counts.items()), columns=['Genre', 'Count'])
genre_counts_df['Percentage'] = round((genre_counts_df['Count'] * 100) / 3322, 3)
genre_counts_df = genre_counts_df.sort_values(by='Count', ascending=False)
genre_counts_df.reset_index(drop=True, inplace=True)

genre_counts_df

,Genre,Count,Percentage
0,Rock,806,24.262
1,Pop,805,24.232
2,Hip Hop,647,19.476
3,Country,395,11.890
4,RnB,222,6.683
5,Electronic,161,4.846
6,Show Tunes,154,4.636
7,Soul,131,3.943
8,Metal,125,3.763
9,Jazz,79,2.378


In [566]:
value_counts = df['big_genres'].value_counts(dropna=False)
value_counts_df = value_counts.reset_index()
value_counts_df['Percentage'] = round((value_counts_df['count'] * 100) / 3322, 3)
value_counts_df


,big_genres,count,Percentage
0,Hip Hop,552,16.616
1,Rock,524,15.774
2,Pop,486,14.630
3,Country,368,11.078
4,Show Tunes,142,4.275
...,...,...,...
136,"Funk, Hip Hop, RnB, Soul",1,0.030
137,"Latin, Pop",1,0.030
138,"Funk, Hip Hop, Rap, Soul",1,0.030
139,"Hip Hop, Jazz",1,0.030


In [567]:
from utils_dataset.location_processor import LocationProcessor

df['location'] = df['artist_country'].apply(LocationProcessor.get_country_name)


value_counts = df['location'].value_counts(dropna=False)
value_counts_df = value_counts.reset_index()
value_counts_df['Percentage'] = round((value_counts_df['count'] * 100) / 3322, 3)
value_counts_df

,location,count,Percentage
0,United States,2355,70.891
1,None,272,8.188
2,United Kingdom,222,6.683
3,Germany,170,5.117
4,Canada,114,3.432
5,Australia,39,1.174
6,Ireland,23,0.692
7,"Korea, Republic of",18,0.542
8,Puerto Rico,16,0.482
9,Italy,12,0.361
